## Setup

In [4]:
import sys
from pathlib import Path

from aocd import get_data, submit

current_day is only available in December (EST)


In [5]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [6]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=18)

## Part a

In [8]:
# Imports
from collections import defaultdict
from heapq import heappop, heappush

In [ ]:
# Functions
def parse_corrupted_locations(corrupted_locations_str: str, n: int = 1024) -> list[complex]:
    """Parse first n locations into complex coordinates."""
    return [complex(*map(int, line.split(","))) for line in corrupted_locations_str.splitlines()[:n]]


def construct_grid(corrupted_locations_str: str, n: int = 1024, max_coordinate: int = 70) -> dict[complex, str]:
    """Construct square grid with the first n corrupted locations marked as '#' and empty locations marked as '.'.

    Note that the coordinates are 0-indexed, so the grid size is maximum coordinate + 1.
    """
    # Parse corrupted locations
    corrupted_locations = parse_corrupted_locations(corrupted_locations_str, n)

    # Construct grid
    return {
        x + y * 1j: "#" if (x + y * 1j) in corrupted_locations else "."
        for y in range(max_coordinate + 1)
        for x in range(max_coordinate + 1)
    }


def find_shortest_path(
    grid: dict[complex, str],
    start: complex,
    end: complex,
) -> int | None:
    """Find shortest path length using A* search. Returns None if no path is found."""
    minimum_step_count_per_position = defaultdict(lambda: float("inf"))
    queue = [(0, t := 0, start)]  # Step count, tie-breaker, position

    while queue:
        # Get next step
        step_count, _, position = heappop(queue)

        # Found end, return step count
        if position == end:
            return step_count

        # Skip if this step_count is worse than previously found
        if step_count >= minimum_step_count_per_position[position]:
            continue

        # Otherwise, track step count for this position
        minimum_step_count_per_position[position] = step_count

        for direction in [1j, 1, -1j, -1]:
            # Find new position
            new_position = position + direction

            # Skip if new position is invalid (wall or out of bounds)
            if grid.get(new_position, "#") == "#":
                continue

            # Increase tie-breaker for complex positions
            t += 1

            heappush(queue, (step_count + 1, t, new_position))

    return None


def find_shortest_path_between_corrupted_locations(data: str, n: int = 1024, max_coordinate: int = 70) -> int | None:
    """Find minimum steps between corrupted locations."""
    # Find corrupted locations
    grid = construct_grid(data, n, max_coordinate)

    # Find start and end positions
    start, end = complex(0, 0), complex(max_coordinate, max_coordinate)

    # Find the shortest path length from start to end, avoiding corrupted locations
    return find_shortest_path(grid, start, end)

In [11]:
# Find the shortest path between corrupted locations
minimum_step_count = find_shortest_path_between_corrupted_locations(data)

In [13]:
# Submit answer
submit(minimum_step_count, part="a", day=18, year=2024)

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part b

In [44]:
# Functions
def find_breaking_point(data: str, start_n: int = 1024) -> str | None:
    """Find first byte that breaks path."""
    corrupted_bytes = data.splitlines()
    left = start_n
    right = len(corrupted_bytes)

    # Binary search for first invalid n
    while left < right:
        mid = (left + right) // 2
        if find_shortest_path_between_corrupted_locations(data, n=mid) is not None:
            left = mid + 1  # Path exists, first invalid must be higher
        else:
            right = mid  # No path, this might be first invalid

    # Return first invalid byte, which is one lower in the list because it is 0-indexed
    return corrupted_bytes[left - 1] if left > start_n else None

In [45]:
# Find first invalid byte
first_invalid_byte = find_breaking_point(data)

In [46]:
# Submit answer
submit(first_invalid_byte, part="b", day=18, year=2024)

aocd will not submit that answer again. At 2025-01-22 18:20:41.789481-05:00 you've previously submitted 60,37 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 18! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
